# Query by provenance and load dobjects

In [ ]:
import lamindb as db
import seaborn as sns
import sqlmodel as sqm

db.header()

Assume we're a different user who wants to continue the work of `test-user1` on the iris dataset.

In [ ]:
!lndb login test-user2

Let's see whether we can find our friend test-user1.

In [ ]:
db.do.query.user(handle="test-user1")

For simple frequent queries, LaminDB has a high-level query API around SQLModel.[^sqlmodel]

[^sqlmodel]: We'll also show how to use SQLModel for arbitrarily complicated queries.

In [ ]:
dobjects = db.do.query.dobject(name="iris")

The high-level query API always returns a list of results:

In [ ]:
dobjects

In [ ]:
dobject = dobjects[0]

To load a dobject into memory, use:

In [ ]:
df = db.do.load.dobject(dobject)

In [ ]:
g = sns.PairGrid(df, hue="target")
g.map_upper(sns.scatterplot, s=15)
g.map_diag(sns.histplot)
g.add_legend();

During your analysis, you notice something strange, and you'd like to understand where the data actually came from.

Any dobject has a data source in form of a `dsource_id`.

In [ ]:
dobject.dsource_id

Let's query for it via a SQL session:

In [ ]:
with db.session() as session:
    dtransform = session.get(db.schema.core.dtransform, dobject.dsource_id)

Inspecting the result we see that the dobject originates from a Jupyter Notebook.

In [ ]:
dtransform

In [ ]:
with db.session() as session:
    jupynb = session.get(
        db.schema.core.jupynb, (dtransform.jupynb_id, "1")
    )  # it's version 1! see jupynb_v

In [ ]:
jupynb

Finally we can identify the person who created the notebook!

In [ ]:
with db.session() as session:
    user = session.get(db.schema.core.user, jupynb.user_id)

In [ ]:
user